In [12]:
# Rolling-window temporal cross-validation for T_log (pipelines A/B/C)
# - Windows: yearly and 3-year rolling (configurable)
# - For each window: aggregate by city (mean), run pipelines A/B/C, compute d_part, d_pca90, d_est, T_log
# - Produce series CSVs, plots, heatmaps, and a Markdown report; update logs/summary
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.covariance import MinCovDet
from scipy import stats

# Configurable parameters
SEED = 42
np.random.seed(SEED)
DATA_FP = os.path.join('data', 'urban_climate.csv')
RESULTS_DIR = 'results'
DETAIL_DIR = os.path.join(RESULTS_DIR, 'temporal_cv')
LOGS_CSV = os.path.join('logs', 'logs.csv')
SUMMARY_MD = os.path.join('logs', 'summary.md')
os.makedirs(DETAIL_DIR, exist_ok=True)

# Rolling window definitions
WINDOWS = [
    {'name': 'annual', 'width_years': 1, 'step_months': 12},
    {'name': '3yr', 'width_years': 3, 'step_months': 6}
]
MIN_YEARS_REQUIRED = 1  # minimum span (years) to process a window

# Locked pipeline params (keeps consistency)
PARAMS_FP = os.path.join(RESULTS_DIR, 'params.json')
if os.path.exists(PARAMS_FP):
    with open(PARAMS_FP, 'r', encoding='utf-8') as f:
        locked = json.load(f)
    FEATURES = locked.get('features', ['temperature_celsius','humidity_percent','precipitation_mm','wind_speed_ms','urban_heat_island_intensity'])
    AGG_COLS = locked.get('aggregation', {}).get('aggregation_group', ['city','country','latitude','longitude'])
    D_CLIP_MIN = locked.get('sanitization', {}).get('d_clip_min', 0.1)
    D_CLIP_MAX = locked.get('sanitization', {}).get('d_clip_max', 100.0)
else:
    FEATURES = ['temperature_celsius','humidity_percent','precipitation_mm','wind_speed_ms','urban_heat_island_intensity']
    AGG_COLS = ['city','country','latitude','longitude']
    D_CLIP_MIN, D_CLIP_MAX = 0.1, 100.0

def append_log(level, message):
    ts = datetime.utcnow().isoformat() + 'Z'
    row = {'timestamp': ts, 'level': level, 'message': message}
    try:
        if os.path.exists(LOGS_CSV):
            logs = pd.read_csv(LOGS_CSV)
            logs = pd.concat([logs, pd.DataFrame([row])], ignore_index=True)
        else:
            logs = pd.DataFrame([row])
        logs.to_csv(LOGS_CSV, index=False)
    except Exception:
        pass
    with open(SUMMARY_MD, 'a', encoding='utf-8') as f:
        f.write(f'\n- {ts} {level}: {message}\n')

def participation_ratio(X):
    cov = np.cov(X, rowvar=False)
    eig = np.linalg.eigvalsh(cov)
    eig = np.maximum(eig, 0.0)
    s = np.sum(eig)
    return 0.0 if s<=0 else (s**2)/np.sum(eig**2)

def d_pca90_from_X(X):
    pca = PCA(n_components=min(X.shape[0], X.shape[1]))
    pca.fit(X)
    cum = np.cumsum(pca.explained_variance_ratio_)
    return int(np.searchsorted(cum, 0.90) + 1) if cum[-1] >= 0.90 else pca.n_components_

def compute_metrics_from_matrix(X, n_eff):
    d_part = participation_ratio(X)
    d_pca90 = d_pca90_from_X(X)
    d_est = (d_part + d_pca90) / 2.0
    d_clipped = float(np.clip(d_est, D_CLIP_MIN, D_CLIP_MAX))
    T_log = (d_clipped - 4.0) * np.log(max(2, int(n_eff)))
    return {'d_part': float(d_part), 'd_pca90': int(d_pca90), 'd_est': float(d_est), 'd_clipped': d_clipped, 'T_log': float(T_log)}

def build_pipeline_matrices(city_df):
    X_raw = city_df[FEATURES].values.astype(float)
    mats = {}
    # A: StandardScaler
    mats['A'] = StandardScaler().fit_transform(X_raw)
    # B: Winsor 1%/99% then StandardScaler
    Xw = X_raw.copy()
    for j in range(Xw.shape[1]):
        lo = np.quantile(Xw[:,j], 0.01)
        hi = np.quantile(Xw[:,j], 0.99)
        Xw[:,j] = np.clip(Xw[:,j], lo, hi)
    mats['B'] = StandardScaler().fit_transform(Xw)
    # C: MinCovDet projection (robust multivariate) fallback to Std
    try:
        Xs = StandardScaler().fit_transform(X_raw)
        mcd = MinCovDet().fit(Xs)
        cov = mcd.covariance_
        eigvals, eigvecs = np.linalg.eigh(cov)
        idx = eigvals.argsort()[::-1]
        eigvecs = eigvecs[:, idx]
        mats['C'] = Xs.dot(eigvecs)
    except Exception:
        mats['C'] = mats['A'].copy()
    return mats

# Load data
append_log('INFO', 'Temporal CV started')
df = pd.read_csv(DATA_FP)
required = set(FEATURES + AGG_COLS + ['year','month'])
if not required.issubset(set(df.columns)):
    append_log('ERROR', f'Data missing required columns: {required - set(df.columns)}')
    raise RuntimeError('Missing required columns for temporal CV')

# Prepare timestamp for windowing
df['year'] = df['year'].astype(int)
df['month'] = df['month'].astype(int)
df['ym'] = df['year'] + (df['month']-1)/12.0  # continuous year-month

min_year = int(df['year'].min())
max_year = int(df['year'].max())

report_rows = []

for win in WINDOWS:
    name = win['name']
    width = win['width_years']
    step = win['step_months'] / 12.0
    start = min_year
    end = max_year - width + 1
    if end < start:
        append_log('WARNING', f'Window {name} not applicable: data span too short')
        continue
    window_centers = []
    t = start
    while t <= max_year - width + 1e-9:
        window_centers.append(round(t, 6))
        t += step
    # results per window for pipelines
    rows_for_window = []
    for wc in window_centers:
        w_start = wc
        w_end = wc + width - 1/365.0  # include full last year approx
        mask = (df['ym'] >= w_start) & (df['ym'] < w_end + 1/12.0)
        df_win = df[mask].copy()
        if df_win.empty:
            continue
        # aggregate per city (mean) as locked
        df_win['city_key'] = df_win[AGG_COLS].astype(str).agg('_'.join, axis=1)
        city_df = df_win.groupby('city_key')[FEATURES].mean().reset_index()
        n_cities = city_df.shape[0]
        if n_cities < 2:
            append_log('WARNING', f'Window {name} center {wc}: insufficient cities ({n_cities}), skipping')
            continue
        mats = build_pipeline_matrices(city_df)
        for pid, Xs in mats.items():
            metrics = compute_metrics_from_matrix(Xs, n_cities)
            row = {'window_type': name, 'window_center': wc, 'pipeline': pid, 'n_cities': n_cities,
                   'd_part': metrics['d_part'], 'd_pca90': metrics['d_pca90'],
                   'd_est': metrics['d_est'], 'd_clipped': metrics['d_clipped'], 'T_log': metrics['T_log']}
            rows_for_window.append(row)
    if rows_for_window:
        df_windows = pd.DataFrame(rows_for_window)
        out_csv = os.path.join(DETAIL_DIR, f'temporal_cv_{name}.csv')
        df_windows.to_csv(out_csv, index=False)
        # Plot T_log time series per pipeline
        plt.figure(figsize=(8,3))
        for pid in df_windows['pipeline'].unique():
            sub = df_windows[df_windows['pipeline']==pid].sort_values('window_center')
            plt.plot(sub['window_center'], sub['T_log'], marker='o', label=pid)
        plt.axhline(0, color='k', linestyle='--')
        plt.xlabel('window center (year)')
        plt.ylabel('T_log')
        plt.title(f'T_log series ({name} windows)')
        plt.legend()
        png = os.path.join(DETAIL_DIR, f'temporal_cv_{name}_Tlog.png')
        plt.tight_layout()
        plt.savefig(png, dpi=150)
        plt.close()
        # Heatmap regime (Saturation/Equilibre/Divergence) per window x pipeline
        df_windows['regime'] = df_windows['T_log'].apply(lambda v: 'S' if v>0 else ('E' if np.isclose(v,0.0,atol=1e-8) else 'D'))
        pivot = df_windows.pivot(index='window_center', columns='pipeline', values='T_log')
        heat_png = os.path.join(DETAIL_DIR, f'temporal_cv_{name}_heatmap.png')
        plt.figure(figsize=(6, max(2, 0.3*len(pivot.index))))
        im = plt.imshow(pivot.values, aspect='auto', cmap='RdBu', vmin=-np.max(np.abs(pivot.values)), vmax=np.max(np.abs(pivot.values)))
        plt.colorbar(im, label='T_log')
        plt.yticks(range(len(pivot.index)), [f"{v:.2f}" for v in pivot.index])
        plt.xticks(range(len(pivot.columns)), pivot.columns)
        plt.title(f'Heatmap T_log ({name})')
        plt.tight_layout()
        plt.savefig(heat_png, dpi=150)
        plt.close()
        # Stability metrics
        stability = []
        for pid in df_windows['pipeline'].unique():
            sub = df_windows[df_windows['pipeline']==pid]
            sign_same = (np.sign(sub['T_log']) == np.sign(df_windows[df_windows['pipeline']==pid]['T_log'].median())).mean()
            stability.append({'window_type': name, 'pipeline': pid, 'windows_count': sub.shape[0], 'sign_consistency_frac': float(sign_same)})
        st_df = pd.DataFrame(stability)
        st_df.to_csv(os.path.join(DETAIL_DIR, f'temporal_cv_{name}_stability.csv'), index=False)
        report_rows.append({'window_type': name, 'csv': out_csv, 'tlog_png': png, 'heatmap_png': heat_png,
                            'stability_csv': os.path.join(DETAIL_DIR, f'temporal_cv_{name}_stability.csv')})
        append_log('INFO', f'Temporal CV {name} complete: saved {out_csv}')
    else:
        append_log('WARNING', f'No windows computed for {name}')

# Comparative report markdown
report_md = os.path.join(RESULTS_DIR, 'temporal_cv_report.md')
with open(report_md, 'w', encoding='utf-8') as f:
    f.write('# Temporal cross-validation report\n\n')
    f.write(f'Generated: {datetime.utcnow().isoformat()}Z\n\n')
    for r in report_rows:
        f.write(f"## Window type: {r['window_type']}\n\n")
        f.write(f"- windows CSV: {r['csv']}\n")
        f.write(f"- T_log timeseries PNG: {r['tlog_png']}\n")
        f.write(f"- T_log heatmap PNG: {r['heatmap_png']}\n")
        f.write(f"- stability CSV: {r['stability_csv']}\n\n")
    f.write('\n## Notes\n\n- Parameters: windows config in the executed cell. See results/params.json for pipeline params if present.\n')

append_log('INFO', f'Temporal CV finished; report saved to {report_md}')
print('Temporal cross-validation finished.')
print('Report:', report_md)
print('Detail folder:', DETAIL_DIR)


C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\1351723274.py:48: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
c:\Users\zackd\anaconda3\Lib\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-3.650637930711747 > -75.356117505982709). You may want to try with a higher value of support_fraction (current value: 0.650).
  warnings.warn(
c:\Users\zackd\anaconda3\Lib\site-packages\sklearn\covariance\_robust_covariance.py:186: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-3.650637930711747 > -75.356117505982709). You may want to try with a higher value of support_fraction (current value: 0.650).
  warnings.warn(
c:\Users\zackd\anaconda3\Li

Temporal cross-validation finished.
Report: results\temporal_cv_report.md
Detail folder: results\temporal_cv


C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\1351723274.py:48: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\1351723274.py:212: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f.write(f'Generated: {datetime.utcnow().isoformat()}Z\n\n')
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\1351723274.py:48: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'


### Résumé synthétique des résultats temporels

- Résultat central : T_log reste négatif sur la très grande majorité des fenêtres temporelles pour les trois pipelines A/B/C, donc le signe T_log < 0 est robuste dans le temps (convergence entre pipelines)【temporal_cv_3yr.csv】【temporal_cv_annual.csv】.  
- Stabilité par fenêtre : pour les fenêtres 3‑ans la **consistence du signe** est très haute (≈ 97.65% pour A/B/C)；pour les fenêtres annuelles elle est plus faible mais élevée (A/C ≈ 88.9%, B ≈ 93.3%)【temporal_cv_3yr_stability.csv】【temporal_cv_annual_stability.csv】.  
- Comportement temporel observé : T_log fluctue dans le temps, avec épisodes isolés où la médiane de la fenêtre s’approche ou dépasse 0 (ex. années autour de 1984, 2003, 2008, 2012, 2014 selon heatmaps), mais ces épisodes sont rares et documentés dans les CSV produits【temporal_cv_3yr.csv】【temporal_cv_annual.csv】.  
- Conclusion pratique : la preuve empirique que T_log < 0 est cohérente et répétable sous les pipelines transparents (A, B, C). La conclusion reste sensible à choix méthodologiques et à certaines périodes historiques — transparence et publication des diagnostics sont donc indispensables.

---

### Interprétation statistique clé (chiffrée)
- 3‑ans rolling windows : sign_consistency_frac = 0.9765 (85 fenêtres) — très stable【temporal_cv_3yr_stability.csv】.  
- Annual windows : sign_consistency_frac ≈ 0.89 (A,C) et 0.933 (B) — stable mais plus variable année à année【temporal_cv_annual_stability.csv】.  
- Implication : en fenêtres larges (3 ans) le résultat est hautement reproductible ; en fenêtres courtes (1 an) il reste majoritairement reproductible mais révèle périodes de bascule utiles à documenter.

---

### Action recommandée immédiate (priorité unique)
Publier côte‑à‑côte : (1) la méthode verrouillée (params.json, README_method.md), (2) feature_quality_report.md, (3) pipelines_comparative_report.md, (4) temporal_cv_report.md et les CSV/plots associés. Justification : cela montre la reproductibilité, documente la sensibilité temporelle et fournit aux relecteurs tout le matériel nécessaire pour auditer votre assertion T_log < 0.

---




Cellule Python — Génération de l’annexe « périodes atypiques » (CSV + diagnostics courts par fenêtre)

 La cellule :

lit les résultats temporels produits (results/temporal_cv/*.csv) ;

identifie fenêtres atypiques où T_log >= 0 ou où le signe de T_log diffère de la médiane temporelle du pipeline ;

pour chaque fenêtre atypique produit un diagnostic court : n_cities, d_part, d_pca90, d_est, T_log, regime, proportion de villes avec Mahalanobis élevée (si disponible), et coverage (nombre d'observations par ville dans la fenêtre) ;

sauve un CSV consolidé results/temporal_cv_atypical_periods.csv et un rapport Markdown results/temporal_cv_atypical_periods.md ; génère aussi un petit plot indiquant les fenêtres atypiques sur la série 3‑yr.

In [13]:
# Génération annexe "périodes atypiques"
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

RESULTS_DIR = 'results'
TCV_DIR = os.path.join(RESULTS_DIR, 'temporal_cv')
OUT_CSV = os.path.join(RESULTS_DIR, 'temporal_cv_atypical_periods.csv')
OUT_MD = os.path.join(RESULTS_DIR, 'temporal_cv_atypical_periods.md')
OUT_PNG = os.path.join(RESULTS_DIR, 'temporal_cv_atypical_periods_overview.png')
DATA_FP = os.path.join('data','urban_climate.csv')

os.makedirs(TCV_DIR, exist_ok=True)

def regime_label(t):
    return 'Saturation' if t>0 else ('Equilibre' if np.isclose(t,0.0,atol=1e-8) else 'Divergence')

# Load all temporal cv CSVs (annual and 3yr)
tcv_files = sorted(glob.glob(os.path.join(TCV_DIR, 'temporal_cv_*.csv')))
if not tcv_files:
    raise RuntimeError('Aucun fichier temporal_cv trouvé dans ' + TCV_DIR)

df_list = []
for fp in tcv_files:
    df = pd.read_csv(fp)
    df['source_file'] = os.path.basename(fp)
    df_list.append(df)
tcv = pd.concat(df_list, ignore_index=True)

# Identify pipeline medians (to detect sign flips relative to long-run behavior)
medians = tcv.groupby('pipeline')['T_log'].median().to_dict()

# Criteria for atypical:
# 1) T_log >= 0 (non-negative) OR
# 2) sign(T_log) != sign(median)  (flip)
def is_atypical(row):
    if row['T_log'] >= 0.0:
        return True
    med = medians.get(row['pipeline'], 0.0)
    return np.sign(row['T_log']) != np.sign(med)

tcv['is_atypical'] = tcv.apply(is_atypical, axis=1)

# Filter atypical windows
atyp = tcv[tcv['is_atypical']].copy()
if atyp.empty:
    # still create empty outputs with note
    pd.DataFrame([]).to_csv(OUT_CSV, index=False)
    with open(OUT_MD, 'w', encoding='utf-8') as f:
        f.write('# Temporal CV - Périodes atypiques\n\n')
        f.write(f'Generated: {datetime.utcnow().isoformat()}Z\n\n')
        f.write('Aucune fenêtre atypique détectée selon les critères (T_log >= 0 ou inversion de signe par rapport à la médiane)\n')
    print('Aucune fenêtre atypique détectée. Fichiers créés (vides/report) :', OUT_CSV, OUT_MD)
else:
    # Read raw data to compute coverage / city count per window for diagnostics
    df_raw = pd.read_csv(DATA_FP)
    # prepare continuous ym
    df_raw['year'] = df_raw['year'].astype(int)
    df_raw['month'] = df_raw['month'].astype(int)
    df_raw['ym'] = df_raw['year'] + (df_raw['month']-1)/12.0

    rows = []
    for idx, r in atyp.sort_values(['window_type','window_center','pipeline']).iterrows():
        wtype = r['window_type']
        wc = float(r['window_center'])
        # reconstruct window width from filename
        if 'annual' in r['source_file'] or wtype=='annual':
            width = 1.0
        elif '3yr' in r['source_file'] or wtype=='3yr':
            width = 3.0
        else:
            width = 1.0
        w_start = wc
        w_end = wc + width - 1/365.0
        mask = (df_raw['ym'] >= w_start) & (df_raw['ym'] < w_end + 1/12.0)
        df_win = df_raw[mask].copy()
        # city coverage: number cities, median obs per city, min obs per city
        if not df_win.empty:
            df_win['city_key'] = df_win[['city','country','latitude','longitude']].astype(str).agg('_'.join, axis=1)
            city_counts = df_win['city_key'].value_counts()
            n_cities = int(city_counts.size)
            median_obs_city = float(city_counts.median())
            min_obs_city = int(city_counts.min())
            # proportion of cities present relative to baseline (max distinct cities in full period)
            total_unique_cities_full = df_raw.groupby(df_raw[['city','country','latitude','longitude']].astype(str).agg('_'.join, axis=1)).ngroups
            coverage_frac = float(n_cities) / max(1, total_unique_cities_full)
        else:
            n_cities = 0
            median_obs_city = np.nan
            min_obs_city = np.nan
            coverage_frac = 0.0

        # Mahalanobis / multivariate outlier signal: try read previous file if present
        md_score = None
        md_fp = os.path.join(RESULTS_DIR, 'feature_mahalanobis_scores.csv')
        if os.path.exists(md_fp):
            try:
                md_df = pd.read_csv(md_fp)
                high_md = (md_df['mahalanobis'] > md_df['mahalanobis'].quantile(0.975)).sum()
                prop_high_md = float(high_md) / max(1, md_df.shape[0])
            except Exception:
                prop_high_md = np.nan
        else:
            prop_high_md = np.nan

        # assemble diagnostic
        rows.append({
            'window_type': r['window_type'],
            'window_center': wc,
            'pipeline': r['pipeline'],
            'n_cities_window': n_cities,
            'median_obs_per_city': median_obs_city,
            'min_obs_per_city': min_obs_city,
            'coverage_frac_vs_full': coverage_frac,
            'd_part': float(r.get('d_part', np.nan)),
            'd_pca90': int(r.get('d_pca90', np.nan)) if not pd.isna(r.get('d_pca90', np.nan)) else np.nan,
            'd_est': float(r.get('d_est', np.nan)),
            'T_log': float(r.get('T_log', np.nan)),
            'regime': regime_label(float(r.get('T_log', np.nan))),
            'prop_high_mahalanobis_cities': prop_high_md,
            'source_file': r['source_file']
        })

    out_df = pd.DataFrame(rows)
    out_df.sort_values(['window_type','window_center','pipeline'], inplace=True)
    out_df.to_csv(OUT_CSV, index=False)

    # Markdown brief report
    with open(OUT_MD, 'w', encoding='utf-8') as f:
        f.write('# Temporal CV - Annexes Périodes atypiques\n\n')
        f.write(f'Generated: {datetime.utcnow().isoformat()}Z\n\n')
        f.write('Critères : T_log >= 0 ou inversion du signe par rapport à la médiane temporelle du pipeline\n\n')
        f.write(f'Total fenêtres détectées : {len(out_df)}\n\n')
        f.write('## Tableau synthétique (extrait)\n\n')
        f.write(out_df.head(50).to_markdown(index=False))
        f.write('\n\n## Observations rapides\n\n')
        f.write('- Les fenêtres listées ci-dessus nécessitent vérification de la couverture et des outliers. Voir le CSV complet pour tous les détails.\n')
        f.write('- Le champ prop_high_mahalanobis_cities donne la proportion (sur l’ensemble des villes) ayant score Mahalanobis élevé dans l\'analyse multivariée ; valeur NaN si non disponible.\n\n')
        f.write('## Fichiers produits\n\n')
        f.write(f'- CSV complet : {OUT_CSV}\n')
        f.write(f'- Raport court (this file) : {OUT_MD}\n\n')

    # Small overview plot: 3-yr T_log series with markers for atypical windows
    try:
        df3 = pd.read_csv(os.path.join(TCV_DIR, 'temporal_cv_3yr.csv'))
        plt.figure(figsize=(10,3))
        for pid, col in zip(['A','B','C'], ['C0','C1','C2']):
            sub = df3[df3['pipeline']==pid].sort_values('window_center')
            plt.plot(sub['window_center'], sub['T_log'], marker='o', color=col, label=pid, linewidth=0.9)
        # mark atypical windows centers
        at_centers = out_df['window_center'].unique()
        for c in sorted(at_centers):
            plt.axvline(c, color='k', alpha=0.12)
        plt.axhline(0, color='k', linestyle='--')
        plt.xlabel('window center (year)')
        plt.ylabel('T_log')
        plt.title('T_log (3yr) with atypical windows marked')
        plt.legend()
        plt.tight_layout()
        plt.savefig(OUT_PNG, dpi=150)
        plt.close()
    except Exception:
        pass

    print('Annexe périodes atypiques générée :', OUT_CSV, OUT_MD, OUT_PNG)


C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\1436546683.py:134: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f.write(f'Generated: {datetime.utcnow().isoformat()}Z\n\n')


Annexe périodes atypiques générée : results\temporal_cv_atypical_periods.csv results\temporal_cv_atypical_periods.md results\temporal_cv_atypical_periods_overview.png


Cellule Python — Export des tables d’observations par fenêtre atypique

 Cette cellule :

lit results/temporal_cv_atypical_periods.csv ;

pour chaque fenêtre atypique valide (window_center non vide) extrait les observations source dans la fenêtre (selon annual ou 3yr) ;

produit pour chaque fenêtre un CSV : results/temporal_cv_atypical_periods_window_<window_center>_<pipeline>.csv contenant les lignes brutes et colonnes additionnelles (city_key, counts par ville) et un résumé par ville (n_obs, mean des features) sauvegardé à côté ;

met à jour logs/summary.md et affiche un résumé final.

In [14]:
# Export per-window raw observations for atypical windows
import os, glob, re
import numpy as np
import pandas as pd
from datetime import datetime

RESULTS_DIR = 'results'
TCV_ATYP_FP = os.path.join(RESULTS_DIR, 'temporal_cv_atypical_periods.csv')
DATA_FP = os.path.join('data', 'urban_climate.csv')
OUT_DIR = os.path.join(RESULTS_DIR, 'temporal_cv_atypical_windows')
LOG_SUMMARY = os.path.join('logs', 'summary.md')

os.makedirs(OUT_DIR, exist_ok=True)

def safe_name(s):
    return re.sub(r'[^0-9A-Za-z_.-]', '_', str(s))

def append_summary(msg):
    ts = datetime.utcnow().isoformat() + 'Z'
    with open(LOG_SUMMARY, 'a', encoding='utf-8') as f:
        f.write(f'\n- {ts} INFO: {msg}\n')

if not os.path.exists(TCV_ATYP_FP):
    raise FileNotFoundError(f'File not found: {TCV_ATYP_FP}')

if not os.path.exists(DATA_FP):
    raise FileNotFoundError(f'Raw data file not found: {DATA_FP}')

tcv = pd.read_csv(TCV_ATYP_FP)
# Drop rows without numeric window_center
tcv_valid = tcv[~tcv['window_center'].isna()].copy()
if tcv_valid.empty:
    print('Aucune fenêtre atypique avec window_center numérique trouvée. Rien à exporter.')
    append_summary('Aucun export — pas de fenêtres atypiques numériques.')
else:
    df_raw = pd.read_csv(DATA_FP)
    # ensure year/month exist and numeric
    if 'year' not in df_raw.columns or 'month' not in df_raw.columns:
        raise RuntimeError('data must contain year and month columns')
    df_raw['year'] = df_raw['year'].astype(int)
    df_raw['month'] = df_raw['month'].astype(int)
    df_raw['ym'] = df_raw['year'] + (df_raw['month'] - 1) / 12.0
    # prepare city_key consistent with pipeline
    agg_cols = ['city','country','latitude','longitude']
    for c in agg_cols:
        if c not in df_raw.columns:
            raise RuntimeError(f'Missing aggregation column in raw data: {c}')
    df_raw['city_key'] = df_raw[agg_cols].astype(str).agg('_'.join, axis=1)

    exported = []
    for _, row in tcv_valid.sort_values(['window_type','window_center','pipeline']).iterrows():
        wtype = str(row.get('window_type', '')).strip()
        wc = float(row['window_center'])
        pipeline = str(row.get('pipeline','')).strip()
        source = str(row.get('source_file',''))
        if 'annual' in wtype.lower():
            width = 1.0
        elif '3' in wtype:
            width = 3.0
        else:
            # fallback: treat as 1 year
            width = 1.0
        w_start = wc
        w_end = wc + width - 1/365.0
        mask = (df_raw['ym'] >= w_start) & (df_raw['ym'] < w_end + 1/12.0)
        df_win = df_raw[mask].copy()
        tag = f"{wtype}_{wc:.2f}_{pipeline}"
        tag_safe = safe_name(tag)
        out_raw_fp = os.path.join(OUT_DIR, f'temporal_cv_atypical_period_window_{tag_safe}_raw.csv')
        out_city_fp = os.path.join(OUT_DIR, f'temporal_cv_atypical_period_window_{tag_safe}_by_city.csv')
        # write raw rows (if empty still create file with header)
        df_win.to_csv(out_raw_fp, index=False)
        # per-city summary: n_obs, mean of numeric features, first/last observation dates
        if not df_win.empty:
            group = df_win.groupby('city_key')
            summary_rows = []
            numeric_cols = df_win.select_dtypes(include=[np.number]).columns.tolist()
            for city, g in group:
                n_obs = int(g.shape[0])
                mean_vals = g[numeric_cols].mean().to_dict()
                first_ym = float(g['ym'].min())
                last_ym = float(g['ym'].max())
                row_out = {'city_key': city, 'n_obs': n_obs, 'first_ym': first_ym, 'last_ym': last_ym}
                # add numeric means (prefix mean_)
                for k,v in mean_vals.items():
                    row_out[f'mean_{k}'] = float(v) if not pd.isna(v) else np.nan
                summary_rows.append(row_out)
            city_df = pd.DataFrame(summary_rows).sort_values('n_obs', ascending=False)
            city_df.to_csv(out_city_fp, index=False)
        else:
            # empty city summary
            pd.DataFrame([]).to_csv(out_city_fp, index=False)
        exported.append({'window_type': wtype, 'window_center': wc, 'pipeline': pipeline,
                         'raw_csv': out_raw_fp, 'city_summary_csv': out_city_fp, 'n_rows': int(df_win.shape[0])})
        append_summary(f'Exported atypical window {tag} -> {os.path.basename(out_raw_fp)} ({int(df_win.shape[0])} rows)')
    # master index CSV
    index_fp = os.path.join(OUT_DIR, 'temporal_cv_atypical_periods_windows_index.csv')
    pd.DataFrame(exported).to_csv(index_fp, index=False)
    print('Export terminé.')
    print('Fichiers exportés (index):', index_fp)
    print(pd.DataFrame(exported).to_string(index=False))
    append_summary(f'Export atypical windows completed; index saved to {index_fp}')


C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\256344354.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\256344354.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\256344354.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\256344354.py:19

Export terminé.
Fichiers exportés (index): results\temporal_cv_atypical_windows\temporal_cv_atypical_periods_windows_index.csv
window_type  window_center pipeline                                                                                          raw_csv                                                                                     city_summary_csv  n_rows
        3yr         1984.0        A    results\temporal_cv_atypical_windows\temporal_cv_atypical_period_window_3yr_1984.00_A_raw.csv    results\temporal_cv_atypical_windows\temporal_cv_atypical_period_window_3yr_1984.00_A_by_city.csv     740
        3yr         1984.0        B    results\temporal_cv_atypical_windows\temporal_cv_atypical_period_window_3yr_1984.00_B_raw.csv    results\temporal_cv_atypical_windows\temporal_cv_atypical_period_window_3yr_1984.00_B_by_city.csv     740
        3yr         1984.0        C    results\temporal_cv_atypical_windows\temporal_cv_atypical_period_window_3yr_1984.00_C_raw.csv    results\tem

C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\256344354.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\256344354.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\256344354.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + 'Z'
C:\Users\zackd\AppData\Local\Temp\ipykernel_5596\256344354.py:19

Actions opérationnelles immédiates recommandées

Inspecter rapidement les fichiers *_by_city.csv pour vérifier si certaines villes présentent n_obs anormalement bas ou moyennes de features extrêmes.

Extraire métadonnées data_source et dates d’installation/maintenance pour les années atypiques listées et joindre les preuves dans l’annexe.

Calculer, par fenêtre atypique et par ville, la fraction d’observations classées outlier (Mahalanobis > 97.5%) et ajouter une colonne summary « flags_for_review » dans l’index.

Cellule Python — enrichir l’index avec proportions d’outliers Mahalanobis et flags

 La cellule :

lit l’index results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_windows_index.csv ;

pour chaque entrée charge le CSV raw correspondant, calcule un score Mahalanobis multivarié sur les colonnes numériques (robuste avec MinCovDet si possible) ;

calcule par ville la proportion de lignes dépassant le seuil 97.5% (empirique) et agrège en métriques fenêtre : proportion d’observations outlier, proportion de villes avec >10% d’outliers, et un flag_for_review simple ;

écrit results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_windows_index_enriched.csv.



In [15]:
# Enrich index with Mahalanobis outlier proportions and review flags
import os
import numpy as np
import pandas as pd
from sklearn.covariance import MinCovDet
from scipy.spatial import distance

BASE_IDX = 'results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_windows_index.csv'
OUT_IDX = 'results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_windows_index_enriched.csv'
NUMERIC_EXCLUDE = ['n_rows']  # numeric cols to ignore if present in raw rows
OUT_DIR = os.path.dirname(OUT_IDX)

os.makedirs(OUT_DIR, exist_ok=True)

def robust_mahalanobis_matrix(X):
    # X: (n_samples, n_features) numeric array
    # returns Mahalanobis distances (n_samples,)
    if X.shape[0] < 2 or X.shape[1] < 1:
        return np.array([np.nan]*X.shape[0])
    try:
        mcd = MinCovDet().fit(X)
        cov = mcd.covariance_
        mean = mcd.location_
    except Exception:
        cov = np.cov(X, rowvar=False)
        mean = np.mean(X, axis=0)
    # ensure invertible
    try:
        inv_cov = np.linalg.inv(cov)
    except Exception:
        # regularize
        cov = cov + np.eye(cov.shape[0]) * 1e-6
        inv_cov = np.linalg.inv(cov)
    diff = X - mean
    md = np.sqrt(np.einsum('ij,jk,ik->i', diff, inv_cov, diff))
    return md

if not os.path.exists(BASE_IDX):
    raise FileNotFoundError(f'Index file not found: {BASE_IDX}')

idx = pd.read_csv(BASE_IDX)

enriched_rows = []
for _, row in idx.iterrows():
    raw_fp = row['raw_csv']
    entry = row.to_dict()
    entry.update({
        'prop_obs_outlier': np.nan,
        'median_md': np.nan,
        'prop_cities_with_gt10pct_outliers': np.nan,
        'max_city_outlier_frac': np.nan,
        'flag_for_review': False,
        'n_rows': int(row['n_rows']) if not pd.isna(row.get('n_rows')) else 0
    })
    if not isinstance(raw_fp, str) or not raw_fp or not os.path.exists(raw_fp):
        enriched_rows.append(entry)
        continue
    try:
        df_raw = pd.read_csv(raw_fp)
        if df_raw.empty:
            enriched_rows.append(entry)
            continue
        # select numeric feature columns for Mahalanobis (exclude identification cols)
        num_cols = df_raw.select_dtypes(include=[np.number]).columns.tolist()
        # remove control columns that don't represent features
        num_cols = [c for c in num_cols if c not in NUMERIC_EXCLUDE and c not in ['year','month','ym']]
        if not num_cols:
            # fallback to attempt specific known features
            fallback = ['temperature_celsius','humidity_percent','precipitation_mm','wind_speed_ms','urban_heat_island_intensity']
            num_cols = [c for c in fallback if c in df_raw.columns]
        if not num_cols:
            enriched_rows.append(entry)
            continue
        X = df_raw[num_cols].astype(float).values
        md = robust_mahalanobis_matrix(X)
        # empirical threshold at 97.5 percentile of md
        thresh = np.nanpercentile(md, 97.5)
        is_out = md >= thresh
        prop_out = float(np.nansum(is_out)) / max(1, X.shape[0])
        entry['prop_obs_outlier'] = prop_out
        entry['median_md'] = float(np.nanmedian(md))
        # per-city proportions: require city_key column, else build from city/country/lat/lon
        if 'city_key' not in df_raw.columns:
            agg_cols = [c for c in ['city','country','latitude','longitude'] if c in df_raw.columns]
            if agg_cols:
                df_raw['city_key'] = df_raw[agg_cols].astype(str).agg('_'.join, axis=1)
            else:
                df_raw['city_key'] = 'unknown'
        city_groups = df_raw.groupby('city_key')
        city_out_fracs = []
        for city, g in city_groups:
            if g.shape[0] == 0:
                continue
            # map rows in g to original indices to index md
            inds = g.index.to_numpy()
            city_frac = float(np.nansum(is_out[inds])) / float(max(1, inds.size))
            city_out_fracs.append(city_frac)
        if city_out_fracs:
            city_out_fracs = np.array(city_out_fracs)
            prop_cities_gt10 = float(np.sum(city_out_fracs > 0.10) / len(city_out_fracs))
            max_city_frac = float(np.nanmax(city_out_fracs))
            entry['prop_cities_with_gt10pct_outliers'] = prop_cities_gt10
            entry['max_city_outlier_frac'] = max_city_frac
        else:
            entry['prop_cities_with_gt10pct_outliers'] = np.nan
            entry['max_city_outlier_frac'] = np.nan
        # Flagging rule (tunable): review if overall outlier prop > 5% OR >10% of cities have >10% outliers OR max city frac > 0.4
        flag = False
        if entry['prop_obs_outlier'] is not None and not np.isnan(entry['prop_obs_outlier']):
            if entry['prop_obs_outlier'] > 0.05:
                flag = True
        if entry['prop_cities_with_gt10pct_outliers'] is not None and not np.isnan(entry['prop_cities_with_gt10pct_outliers']):
            if entry['prop_cities_with_gt10pct_outliers'] > 0.10:
                flag = True
        if entry['max_city_outlier_frac'] is not None and not np.isnan(entry['max_city_outlier_frac']):
            if entry['max_city_outlier_frac'] > 0.40:
                flag = True
        entry['flag_for_review'] = bool(flag)
    except Exception as e:
        # preserve entry with NaNs and continue
        entry['error'] = str(e)
    enriched_rows.append(entry)

enriched_df = pd.DataFrame(enriched_rows)
enriched_df.to_csv(OUT_IDX, index=False)
print('Enriched index written to:', OUT_IDX)
print(enriched_df[['window_type','window_center','pipeline','prop_obs_outlier','prop_cities_with_gt10pct_outliers','max_city_outlier_frac','flag_for_review']].to_string(index=False))


Enriched index written to: results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_windows_index_enriched.csv
window_type  window_center pipeline  prop_obs_outlier  prop_cities_with_gt10pct_outliers  max_city_outlier_frac  flag_for_review
        3yr         1984.0        A          0.025676                               0.05               0.486486             True
        3yr         1984.0        B          0.025676                               0.05               0.513514             True
        3yr         1984.0        C          0.025676                               0.05               0.486486             True
        3yr         2014.0        A          0.025676                               0.05               0.513514             True
        3yr         2014.0        B          0.025676                               0.05               0.513514             True
        3yr         2014.0        C          0.025676                               0.05               0.5

Résumé des enrichissements accomplis

Index enrichi créé : results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_windows_index_enriched.csv.

Colonnes ajoutées et calculées : prop_obs_outlier, median_md, prop_cities_with_gt10pct_outliers, max_city_outlier_frac, flag_for_review.

Règle de flag appliquée : review si prop_obs_outlier > 5% ou proportion de villes ayant >10% outliers > 10% ou max_city_outlier_frac > 40%.

Observations synthétiques : toutes les fenêtres atypiques listées ont flag_for_review = True, prop_obs_outlier ≈ 2.56–2.69% et quelques villes atteignent des fractions d’outliers ≳ 48–54% (max_city_outlier_frac).